**Script for PlanX Description Generator**

Using GPT-oss OpenAI model gpt-oss-20b-unsloth-bnb-4bit from Unsloth

Copied from https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Qwen3_(4B)-Instruct.ipynb#scrollTo=95_Nn-89DhsL

### Installation

In [ ]:
%%capture
import os, re
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    import torch; v = re.match(r"[0-9\.]{3,}", str(torch.__version__)).group(0)
    xformers = "xformers==" + ("0.0.32.post2" if v == "2.8.0" else "0.0.29.post3")
    !pip install --no-deps bitsandbytes accelerate {xformers} peft trl triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets>=3.4.1,<4.0.0" "huggingface_hub>=0.34.0" hf_transfer
    !pip install --no-deps unsloth
!pip install transformers==4.55.4

### Unsloth

`FastModel` supports loading nearly any model now! This includes Vision and Text models!

In [ ]:
from unsloth.chat_templates import CHAT_TEMPLATES
print(list(CHAT_TEMPLATES.keys()))

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
['unsloth', 'zephyr', 'chatml', 'mistral', 'llama', 'vicuna', 'vicuna_old', 'vicuna old', 'alpaca', 'gemma', 'gemma_chatml', 'gemma2', 'gemma2_chatml', 'llama-3', 'llama3', 'phi-3', 'phi-35', 'phi-3.5', 'llama-3.1', 'llama-31', 'llama-3.2', 'llama-3.3', 'llama-32', 'llama-33', 'qwen-2.5', 'qwen-25', 'qwen25', 'qwen2.5', 'phi-4', 'gemma-3', 'gemma3', 'qwen-3', 'qwen3', 'gemma-3n', 'gemma3n', 'gpt-oss', 'gptoss', 'qwen3-instruct', 'qwen3-thinking']


In [ ]:
from unsloth import FastModel
import torch

model, tokenizer = FastModel.from_pretrained(
    model_name = "unsloth/gpt-oss-2e0b-unsloth-bnb-4bit",
    max_seq_length = 2048, # Choose any for long context!
    load_in_4bit = True,  # 4 bit quantization to reduce memory
    load_in_8bit = False, # [NEW!] A bit more accurate, uses 2x memory
    full_finetuning = False, # [NEW!] We have full finetuning now!
    # token = "hf_...", # use one if using gated models
)

==((====))==  Unsloth 2025.10.1: Fast Gpt_Oss patching. Transformers: 4.55.4.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: Using float16 precision for gpt_oss won't work! Using float32.


model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/3.37G [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.00G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/165 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/27.9M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/446 [00:00<?, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

We now add LoRA adapters so we only need to update a small amount of parameters!

In [ ]:
model = FastModel.get_peft_model(
    model,
    r = 32, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 32,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth: Making `model.base_model.model.model` require gradients


<a name="Data"></a>
### Data Prep

In [ ]:
import pandas as pd
from datasets import Dataset

raw_data = pd.read_csv('camden_output_data.csv', index_col=None, header=1, encoding='latin1')
raw_data.dropna(inplace = True)

# we drop first column and change column names to prepare the data in the prompt-completion style
df = raw_data[['ON-LINE APPLICATION', 'DECIDED-FINAL']].rename(
    columns={'ON-LINE APPLICATION': 'prompt', 'DECIDED-FINAL': 'completion'}
)
df

,prompt,completion
1,Replacement of 1 existing cracked window and 2...,Replacement of existing wooden window and door...
2,Provision of 2 (no.) external wall-mounted VRV...,Installation of 2 external wall-mounted air-co...
3,Full Planning: Proposed 2no. new outdoor AC co...,Installation of 2 replacement condenser units ...
4,Lime (T1) - Re-pollard the trees back to most ...,FRONT GARDEN: 2 x Limes (G1) - Re-pollard back...
5,Whitebeam (T1) - Reduce crown back to previous...,FRONT GARDEN: 1 x Whitebeam (T1) - Reduce crow...
...,...,...
17633,This is a Full Planning Application for a rear...,Erection of an outbuilding to the rear garden.
17634,"Installation of a replacement shopfront, inclu...","['Installation of a replacement shopfront, inc..."
17635,Method Statement relating to the repairs of th...,Details of Method Statement relating to the re...
17636,Demolition of existing conservatory and replac...,"['Erection of replacement conservatory, includ..."


In [ ]:
# immediately load the raw data into a Dataset object in the prompt-completion style

ds = Dataset.from_pandas(df)
ds

Dataset({
    features: ['prompt', 'completion', '__index_level_0__'],
    num_rows: 17637
})

In [ ]:
from unsloth.chat_templates import get_chat_template
tokenizer = get_chat_template(
    tokenizer,
    chat_template = 'gpt-oss', #'gpt-oss', 'gptoss'
)

In [ ]:
def format_with_chat_template(example):
    # Create the message structure that the chat template expects
    dct = [
        # "prompt": [
          {"role": "system", "content": "You are a trained planning officer at a local council in the UK. Change the following piece of text, which is a description of a planning application, to give it the best chance of success."},
          {"role": "user", "content": example["prompt"]},
        # ],
        # "completion": [
          {"role": "assistant", "content": example["completion"]},
        # ]
    ]

    # Apply the chat template, returning a single formatted string
    # `tokenize=False` means we get a string, not token IDs. SFTTrainer will tokenize it.
    templated = tokenizer.apply_chat_template(
        conversation=dct,
        tokenize = False,
        add_generation_prompt = False,
    )

    return {"text": templated}

# Use .map() to apply the function to the entire dataset
formatted_dataset = ds.map(format_with_chat_template)

Map:   0%|          | 0/17637 [00:00<?, ? examples/s]

Let's see how row 100 looks like!

In [ ]:
formatted_dataset[100]

{'prompt': 'New roof access, extension of roof terrace floor area, internal alterations',
 'completion': "Non material amendments to planning permission 2020/4960/P (dated 01/03/2021) for 'Creation of roof terrace with associated external alterations' approved, NAMELY to alter location of roof access stair and rooflight.",
 '__index_level_0__': 101,
 'text': "<|start|>system<|message|>You are ChatGPT, a large language model trained by OpenAI.\nKnowledge cutoff: 2024-06\nCurrent date: 2025-10-08\n\nReasoning: medium\n\n# Valid channels: analysis, commentary, final. Channel must be included for every message.<|end|><|start|>developer<|message|># Instructions\n\nYou are a trained planning officer at a local council in the UK. Change the following piece of text, which is a description of a planning application, to give it the best chance of success.<|end|><|start|>user<|message|>New roof access, extension of roof terrace floor area, internal alterations<|end|><|start|>assistant<|channel|>f

In [ ]:
type(formatted_dataset)

datasets.arrow_dataset.Dataset

We now have to apply the chat template for `Qwen-3` onto the conversations, and save it to `text`.

Let's see how the chat template did!


In [ ]:
result = formatted_dataset.train_test_split(test_size=0.2, shuffle=True, seed=42)
result

DatasetDict({
    train: Dataset({
        features: ['prompt', 'completion', '__index_level_0__', 'text'],
        num_rows: 14109
    })
    test: Dataset({
        features: ['prompt', 'completion', '__index_level_0__', 'text'],
        num_rows: 3528
    })
})

In [ ]:
train_ds = result["train"]
test_ds = result["test"]

train_ds[100]

{'prompt': 'Works to the first floor level comprising of removal of non-original wall, reinstatement of wall alterations to modern bathroom',
 'completion': 'Reconfiguration of internal walls at first floor level.',
 '__index_level_0__': 6844,
 'text': '<|start|>system<|message|>You are ChatGPT, a large language model trained by OpenAI.\nKnowledge cutoff: 2024-06\nCurrent date: 2025-10-08\n\nReasoning: medium\n\n# Valid channels: analysis, commentary, final. Channel must be included for every message.<|end|><|start|>developer<|message|># Instructions\n\nYou are a trained planning officer at a local council in the UK. Change the following piece of text, which is a description of a planning application, to give it the best chance of success.<|end|><|start|>user<|message|>Works to the first floor level comprising of removal of non-original wall, reinstatement of wall alterations to modern bathroom<|end|><|start|>assistant<|channel|>final<|message|>Reconfiguration of internal walls at firs

<a name="Train"></a>
### Train the model
Now let's train our model. We do 60 steps to speed things up, but you can set `num_train_epochs=1` for a full run, and turn off `max_steps=None`.

In [ ]:
from trl import SFTTrainer, SFTConfig
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_ds,
    eval_dataset = test_ds, # Can set up evaluation!
    args = SFTConfig(
        dataset_text_field = "text",
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4, # Use GA to mimic batch size!
        warmup_steps = 5,
        num_train_epochs = 1, # Set this for 1 full training run.
        # max_steps = 60,
        learning_rate = 2e-4, # Reduce to 2e-5 for long training runs
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        report_to = "none", # Use this for WandB etc
        eval_strategy="epoch",
    ),
)

Unsloth: Switching to float32 training since model cannot work with float16


Unsloth: Tokenizing ["text"] (num_proc=6):   0%|          | 0/14109 [00:00<?, ? examples/s]

Unsloth: Tokenizing ["text"] (num_proc=6):   0%|          | 0/3528 [00:00<?, ? examples/s]

We also use Unsloth's `train_on_completions` method to only train on the assistant outputs and ignore the loss on the user's inputs. This helps increase accuracy of finetunes!

In [ ]:
from unsloth.chat_templates import train_on_responses_only
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<|start|>system<|message|>",
    response_part = "<|start|>assistant<|channel|>final<|message|>",
)

Map (num_proc=2):   0%|          | 0/14109 [00:00<?, ? examples/s]

Map (num_proc=2):   0%|          | 0/3528 [00:00<?, ? examples/s]

Let's verify masking the instruction part is done! Let's print the 100th row again.

In [ ]:
tokenizer.decode(trainer.train_dataset[100]["input_ids"])

'<|start|>system<|message|>You are ChatGPT, a large language model trained by OpenAI.\nKnowledge cutoff: 2024-06\nCurrent date: 2025-10-08\n\nReasoning: medium\n\n# Valid channels: analysis, commentary, final. Channel must be included for every message.<|end|><|start|>developer<|message|># Instructions\n\nYou are a trained planning officer at a local council in the UK. Change the following piece of text, which is a description of a planning application, to give it the best chance of success.<|end|><|start|>user<|message|>Works to the first floor level comprising of removal of non-original wall, reinstatement of wall alterations to modern bathroom<|end|><|start|>assistant<|channel|>final<|message|>Reconfiguration of internal walls at first floor level.<|end|>'

Now let's print the masked out example - you should see only the answer is present:

In [ ]:
tokenizer.decode([tokenizer.pad_token_id if x == -100 else x for x in trainer.train_dataset[100]["labels"]]).replace(tokenizer.pad_token, " ")

'                                                                                                                                          Reconfiguration of internal walls at first floor level.<|end|>'

In [ ]:
# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.741 GB.
14.441 GB of memory reserved.


Let's train the model! To resume a training run, set `trainer.train(resume_from_checkpoint = True)`

In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 14,109 | Num Epochs = 1 | Total steps = 1,764
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 15,925,248 of 20,930,682,432 (0.08% trained)


Epoch,Training Loss,Validation Loss


Epoch,Training Loss,Validation Loss


In [ ]:
# @title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(
    f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training."
)
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

8013.7364 seconds used for training.
133.56 minutes used for training.
Peak reserved memory = 11.188 GB.
Peak reserved memory for training = 7.372 GB.
Peak reserved memory % of max memory = 75.897 %.
Peak reserved memory for training % of max memory = 50.01 %.


<a name="Inference"></a>
### Inference
Let's run the model via Unsloth native inference! According to the `Qwen-3` team, the recommended settings for instruct inference are `temperature = 0.7, top_p = 0.8, top_k = 20`

For reasoning chat based inference, `temperature = 0.6, top_p = 0.95, top_k = 20`

In [ ]:
messages = [
    {"role" : "user", "content" : "Continue the sequence: 1, 1, 2, 3, 5, 8,"}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize = False,
    add_generation_prompt = True, # Must add for generation
)

from transformers import TextStreamer
_ = model.generate(
    **tokenizer(text, return_tensors = "pt").to("cuda"),
    max_new_tokens = 1000, # Increase for longer outputs!
    temperature = 0.7, top_p = 0.8, top_k = 20, # For non thinking
    streamer = TextStreamer(tokenizer, skip_prompt = True),
)

13<|im_end|>
